In [1]:
import pandas as pd
from pcnn.utils import get_experiment_config, get_sweep_variables, get_all_results_exp, get_best

In [2]:
def extract_results(exp_name,timestamp):
    exp_config = get_experiment_config(exp_name,timestamp)
    sweep_variables = get_sweep_variables(exp_config)
    df = get_all_results_exp(exp_name, timestamp, sweep_variables)
    sweep_variables = {k:v for k,v in sweep_variables.items() if k not in ["data.random_state"]}
    return df, sweep_variables

In [4]:
# df_pointnet, pointnet_sweep_variables = extract_results("pointnet++_modelnet_dense","2023-09-07_21-23-56")
# df_pointnet, pointnet_sweep_variables = extract_results("pointnet++_melanoma_dense","2023-09-11_11-27-41")
# df_pointnet, pointnet_sweep_variables = extract_results("point_transformer_melanoma_dense","2023-09-11_11-27-49")
# df_pointnet, pointnet_sweep_variables = extract_results("pointnet++_tcells_dense","2023-09-18_12-55-19")
df_pointnet, pointnet_sweep_variables = extract_results("legs_melanoma_knn", "2024-01-30_16-32-29")

No PKL file found for legs_melanoma_knn 2024-01-30_16-32-29 7
Config for this run : 
{'original_work_dir': '${hydra:runtime.cwd}', 'seed': None, 'n_seeds': 10, 'name': 'legs_melanoma_knn', 'trainer': {'wandb_user': 'siddharth-viswanath28', 'early_stopping': 20, 'accelerator': 'cpu', 'max_epochs': 100, 'wandb': False, 'gradient_clip_val': 1.0}, 'dataset_name': 'melanoma', 'data': {'_target_': 'pcnn.data.melanoma.MelanomaData', '_partial_': True, 'n_samples': 100, 'batch_size': 32, 'num_workers': 0, 'random_state': 45, 'pin_memory': False, 'graph_construct': '${graph_construct}', 're_precompute': False, 'reprocess_if_different': True, 'train_size': None, 'scattering_n_pca': '${model.scattering_n_pca}'}, 'model_name': 'legs', 'model': {'_target_': 'pcnn.models.pcnn.PCNN', 'num_layers': 1, 'hidden_dim': None, 'lr': 0.01, 'compute_P': False, 'scattering_n_pca': None, 'layer': {'filter_method': 'legs', 'combine_method': 'identity', 'activation': 'identity', 'cross_channel_conv': 'identity', 

In [5]:
configs = [(df_pointnet, pointnet_sweep_variables,"PointNet++", "Raw")]

df =[]
for config in configs:
    df_ = get_best(config[0], config[1])
    df_["Model name"] = config[2]
    df_["Graph type"] = config[3]
    df.append(df_)
    
df = pd.concat(df)
df = df[["Model name", "Graph type", "val_acc","val_acc_std","test_acc", "test_acc_std"]]
    



In [6]:
df["Accuracy"] = df["test_acc"].round(2).astype(str) + " $\pm$ " + df["test_acc_std"].round(2).astype(str)
print(df.pivot(index = "Model name", columns = "Graph type", values = "Accuracy").to_latex(index = True, escape = False))

\begin{tabular}{ll}
\toprule
Graph type & Raw \\
Model name &  \\
\midrule
PointNet++ & 0.71 $\pm$ 0.09 \\
\bottomrule
\end{tabular}

